# Paths in a maze
Many common search problems in computer science can be treated as searching paths within a maze.
Aim of this assignment is devising a proper data structure to model a maze and support various path finding algorithms.
For the purpose of this exercise, the maze will be represented as a two-dimensional grid of cells.
A maze can have both *free* and *blocked* cells. Only free cells can be traversed.
You are free to decide whether cell adjacency applies only to the four main direction or if you want to support also diagonal adjacency.
A path in the maze is a sequence of adjacent traversable cells, starting from a given start position and ending at a given target position.

## Generating a maze
The `Maze` class should keep track of the grid representing its state.
It should also have instance attributes for the number of rows, number of columns, start and goal location. 
The maze should provide an initialisation method to fill it with random blocked cells. The resulting maze should be fairly sparse in order to (almost always) guarantee the existence of a path between two positions. The sparseness can be treated as an additional attribute of an instance.

## Helper methods
It can be useful to have a function that checks whether the visit reached the goal during the search. Regardless of the visit strategy adopted, this is a check that all the algorithms will have to make.

In order to model the act of moving across the maze, it could be useful to define an appropriate method that given a location will return all the available *next* locations reachable from such a position. In this way, it is possible to decouple movement rules from the chosen visit strategies.

These are just suggestion. You may implement any additional helper method you deem suitable, or none at all.

## Visit strategies
The maze should support some of the most common visit strategies. It should support at least BFS and DFS visits. In the following subsection there will be a brief description of various well-known algorithms, but you are free to include any additional one you may wish to experiment with.
The A* search may be a bit trickier to implement correctly if you are unfamiliar with the topic, but it is objectively the most interesting one to play with.

### Depth-first search (DFS)

A [depth-first search](https://en.wikipedia.org/wiki/Depth-first_search) (DFS) is a search that goes as deeply as it can
before backtracking to its last decision point if it reaches a dead end.

The DFS algorithm, in its iterative version, usually relies on a last-in-first-out data structure known as a stack. 
Informally, in a stack the last inserted *item* is placed on top of the stack (`push`) and is the first to be pulled off the stack (`pop`). 
Note that implementing a stack using a Python list is as simple as always appending items onto its right end and always removing items from the same end. 

An in-progress depth-first search needs to keep track of two data structures: the stack of states that we are considering searching and the set of states that we have already searched. As long as there are more states to visit, the DFS will keep checking
whether the current state is a goal and adding its reachable set to the stack. It will also mark each state that has already
been searched as explored to avoid revisiting already seen positions. If the stack is empty, it means there is no state left to search.

If you are familiar with recursion, the DFS has a very natural recursive implementation that does not require a stack.

### Breadth-first search (BFS)
A [breadth-first search](https://en.wikipedia.org/wiki/Breadth-first_search) (BFS) is a search that locally explore the whole neighbourhood at a given depth before moving on. 

Be reminded that a complete implementation of BFS has been already discussed during the last training session of Module 2.

To implement BFS, a first-in-first-out data structure, a queue, is required.
Informally, in a queue the first *item* who got queued is the first one to be dequeued. At a minimum, a queue has
the same `push` and `pop` methods as a stack, and it can rely on the Python `deque` built-in to be implemented.

The BFS algorithm for a breadth-first search is identical to the iterative algorithm for DFS, once the stack is swapped with a queue. Such a swap changes the order in which states are searched and ensures that the states closest to the start state are searched first.

### A* 
[A*](https://en.wikipedia.org/wiki/A*_search_algorithm) (read, a-star) is an optimised search algorithm that aims at finding the shortest path from start state to goal state. A* uses a combination of a cost function and a heuristic function to focus its search on pathways most likely to get to the goal quickly.

The cost function, $g(n)$, evaluates the cost to get to a particular state. For a maze, it would be the number of steps needed to reach a given position. The heuristic function, $h(n)$, gives an estimate of the cost to get from the *current* position to the goal state. In order for the algorithm to produce an optimal solution, the heuristic must be admissible. An admissible heuristic is one that never overestimates the cost to reach the goal. In the case of a two-dimensional plane, one example is a straight-line distance heuristic: a straight line is always the shortest path!

The total cost for any state considered is $f(n) = g(n) + h(n)$. When choosing the next state to explore, A* picks the one with the lowest $f(n)$.

To pick the state with the lowest $f(n)$ you can uses a priority queue. A priority queue keeps its elements in an internal
order, such that the first element popped out is always the highest-priority element.
In the case of A*, the highest-priority item is the one with the lowest $f(n)$.
Python standard library contains `heappush` and `heappop` functions that will manage a list as if it was a binary heap.

#### Heuristics
A heuristic is an *intuition* about a way to solve a given problem.
In this scenario, a heuristic aims to choose the best location to search next in order to reach the goal *efficiently*.

In a way, it is but an educated guess, taking into account the expected *closeness* to the goal, on which move to make next.

Heuristics that calculate smaller values end up leading to a search through more states, whereas heuristics closer to the
exact real distance, without overshooting otherwise they would be inadmissible, lead to a search through fewer states. 
The better the estimate, the more performing the heuristic. 

- As geometry teaches us, the shortest path between two points is a straight line. A straight-line heuristic will always be admissible for our search. The Euclidean distance, derived from the Pythagorean theorem, can be used directly by A* in order to guesstimate the distance from the goal
- If you are supporting movements only along the four main directions, there is a better heuristic you may wish to adopt: the Manhattan distance

## (Optional) Search visualisation
The `pygame` library is a cross-platform set of Python modules designed for writing video games. In its most basic usage, it can be used to produce simple 2D visualisation tools. You can try to integrate it in your solution to visualise the steps taken from the various visits your maze supports.

Note that this is a far more advanced request, so it is <u>completely optional</u>.

In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import random
from collections import deque
import heapq
import pygame 

class Maze:
    
    def __init__(self, rows=10, columns=10, start=(0,0), goal=(9,9), bp=0.1):
        
        self.n_rows=rows
        self.n_cols=columns
        self.start=start
        self.goal=goal
        self.blocked_cells=[(x,y) for x in range(columns) for y in range(rows) if (random.random()<bp and (x,y)!=start and (x,y)!=goal)]
        self.state=[start[0],start[1]]
        
        self.discovered=set() # for DFS and BFS
        self.stop=False # for DFS and BFS
        
        #input
        self.search= input("Select the search algo, then press key when the Pygame window opens to start playing. \n When the game finishes, please wait 5 seconds to exit: \n - 1: DFS\n - 2: BFS\n - 3: A*\n")
        
        # Pygame
        self.width = 500
        self.height = 500
        self.cell_size = min(self.width / self.n_cols, self.height / self.n_rows)
        self.frame_rate=100
        self.well_done=False
        
        self.w = (255,255,255) #grid
        self.k = (0,0,0) #obstacles
        self.b = (0,0,255) #start 
        self.r = (255,0,0) #goal
        self.m = (255,0,255) #path
        
        self.cell_states = np.zeros((self.n_rows, self.n_cols)) #grid
        for X in self.blocked_cells:                                   
            self.cell_states[X[0],X[1]] = 1                     #obstacles
        self.cell_states[self.start[0],self.start[1]] = 2       #start
        self.cell_states[self.goal[0],self.goal[1]] = 3         #goal
        
        pygame.init()
        self.screen = pygame.display.set_mode((self.width, self.height))
        self.screen.fill(self.w)
        pygame.display.set_caption('Search in a Maze')
        self.clock = pygame.time.Clock()
        self.draw_maze()
        self.run=True
        self.pause=True
        
        while True:
            
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.run = False
                    pygame.quit()
                    break    
            
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_SPACE:
                        self.pause = not self.pause
                    
            if not self.run:
                break   
                
            if self.pause:
                continue
            
            self.draw_maze()
            
            if(self.search=='1'):
                self.DFS()
                if self.well_done==False:
                    self.display_sad_message()
                pygame.time.wait(5000)
                pygame.quit()
                break
            elif(self.search=='2'):
                self.BFS()
                if self.well_done==False:
                    self.display_sad_message()
                pygame.time.wait(5000)
                pygame.quit()
                break
            elif(self.search=='3'):
                self.A_star()
                if self.well_done==False:
                    self.display_sad_message()
                pygame.time.wait(5000)
                pygame.quit()
                break
        
#-----------------------------------------------------------------------------------------------------------
    
    def draw_maze(self):

        for row in range(self.n_rows):
            for col in range(self.n_cols):
                x = col * self.cell_size
                y = row * self.cell_size
                rect = pygame.Rect(x, y, self.cell_size, self.cell_size)

                if self.cell_states[row,col] == 1:
                    pygame.draw.rect(self.screen, self.k, rect)
                elif self.cell_states[row,col] == 2:
                    pygame.draw.rect(self.screen, self.b, rect)
                elif self.cell_states[row,col] == 3: 
                    pygame.draw.rect(self.screen, self.r, rect)
                elif self.cell_states[row,col] == 4: 
                    pygame.draw.rect(self.screen, self.m, rect)
        
        #self.clock.tick(self.frame_rate)
        pygame.display.update()
        pass

       
    def display_message(self):
        font = pygame.font.Font('freesansbold.ttf', 80)
        text = font.render('GOAL!', True, (255,255,0), (0,255,255))
        textRect = text.get_rect()
        textRect.center = (self.width // 2, self.height // 2)
        self.screen.blit(text, textRect)
        pygame.display.update()
        self.well_done=True
        pass
    
    def display_sad_message(self):
        font = pygame.font.Font('freesansbold.ttf', 80)
        text = font.render('TRAPPED!', True, (128,0,0), (0,255,255))
        textRect = text.get_rect()
        textRect.center = (self.width // 2, self.height // 2)
        self.screen.blit(text, textRect)
        pygame.display.update()
        self.well_done=True
        pass

        
#-----------------------------------------------------------------------------------------------------------    
    
    def __str__(self):
        return(f'rows: {self.n_rows}, columns: {self.n_cols}, start: {self.start}, goal: {self.goal}, blocked: {self.blocked_cells}, state: {self.state}')   
    
    def PBC(self,step):
        state=np.array(self.state)+step
        if state[0] < 0 or state[0] >= self.n_cols:
            state[0] = state[0] % self.n_cols
        if state[1] < 0 or state[1] >= self.n_rows:
            state[1] = state[1] % self.n_rows
        return (state[0],state[1])       
        
    def next_available(self):
        contour = set()
        for j in (-1,0,1):
            for k in (-1,0,1):
                if not (j==k==0):
                    neighbor=self.PBC(np.array([j,k]))
                    if neighbor not in self.blocked_cells:
                        contour.add(neighbor) 
        return contour    

    def check_goal(self):
        if self.state==[self.goal[0],self.goal[1]]:
            return True 
        
#-----------------------------------------------------------------------------------------------------------        
    
    def DFS(self):
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.run = False
                pygame.quit()
                return

        v=self.state      
        self.discovered.add((v[0],v[1]))
        print(v)
        if not( v == [self.start[0],self.start[1]] or v == [self.goal[0],self.goal[1]] ):
            self.cell_states[v[0],v[1]] = 4
        self.draw_maze()
        
        if self.check_goal():
            print('DFS scored')
            self.display_message()
            self.stop=True 
            
        for w in self.next_available()-self.discovered:
            self.state=[w[0],w[1]]
            
            if self.stop==True:
                self.state=[self.start[0],self.start[1]]
                self.discovered=set()
                return
            else:
                self.DFS()
                
#-----------------------------------------------------------------------------------------------------------                
                
    def BFS(self):
        
        Q=deque()
        root=self.start
        self.discovered.add(root)
        Q.append(root)
        
        while Q:
            
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.run = False
                    pygame.quit()
                    return
            
            v=Q.popleft()
            self.state=[v[0],v[1]]
            print(self.state)
            
            if not( self.state == [self.start[0],self.start[1]] or self.state == [self.goal[0],self.goal[1]] ):
                self.cell_states[self.state[0],self.state[1]] = 4
            self.draw_maze()
            
            if self.check_goal():
                self.state=[self.start[0],self.start[1]]
                self.discovered=set()
                print('BFS scored')
                self.display_message()
                return
            
            for w in self.next_available()-self.discovered:
                self.discovered.add(w)
                Q.append(w) 
                
#-----------------------------------------------------------------------------------------------------------                
    
    #def heuristic(self,x,y): # Euclidean distance
    #    dx=min( (x[0]-y[0]) , min(x[0],y[0])+self.n_cols-max(x[0],y[0]) )
    #    dy=min( (x[1]-y[1]) , min(x[1],y[1])+self.n_rows-max(x[1],y[1]) )
    #    return np.sqrt(dx**2+dy**2)
    
    def heuristic(self,x,y): # Chebyshev distance
        dx=min( abs(x[0]-y[0]) , min(x[0],y[0])+self.n_cols-max(x[0],y[0]) )
        dy=min( abs(x[1]-y[1]) , min(x[1],y[1])+self.n_rows-max(x[1],y[1]) )
        return max(dx,dy)
    
    def A_star(self):
        
        open_set=[]
        heapq.heapify(open_set)
        closed_set = set()
        root = self.start
        g = {root: 0}
        f = {root: self.heuristic(root, self.goal)}
        heapq.heappush(open_set, (f[root], root))
        
        while open_set:
            
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.run = False
                    pygame.quit()
                    return
            
            current = heapq.heappop(open_set)[1]
            self.state = [current[0],current[1]]
            print(self.state)

            if not( self.state == [self.start[0],self.start[1]] or self.state == [self.goal[0],self.goal[1]] ):
                self.cell_states[self.state[0],self.state[1]] = 4
            self.draw_maze()
            
            if self.check_goal():
                print('A* scored')
                self.display_message()
                return

            closed_set.add(current)

            for neighbor in self.next_available() - closed_set:
                tentative_g = g[current] + 1
                if neighbor not in g or tentative_g < g[neighbor]:
                    g[neighbor] = tentative_g
                    f[neighbor] = tentative_g + self.heuristic(neighbor, self.goal)
                    heapq.heappush(open_set, (f[neighbor], neighbor))
        
        
    

pygame 2.1.2 (SDL 2.0.18, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
r=60
c=60
xstart=np.random.randint(0,c)
ystart=np.random.randint(0,r)
xgoal=np.random.randint(0,c)
ygoal=np.random.randint(0,r)
trial_maze=Maze(rows=r, columns=c, start=(xstart,ystart), goal=(xgoal,ygoal), bp=0.5)
#bp is the probability for a cell to be blocked, given it is not the start or goal

Select the search algo, then press key when the Pygame window opens to start playing. 
 When the game finishes, please wait 5 seconds to exit: 
 - 1: DFS
 - 2: BFS
 - 3: A*
 3


[18, 26]
[17, 27]
[16, 28]
[16, 29]
[15, 30]
[18, 27]
[15, 31]
[16, 30]
[17, 29]
[19, 25]
[18, 24]
[17, 24]
[19, 27]
[15, 32]
[17, 30]
[18, 23]
[17, 22]
[16, 22]
[15, 21]
[14, 20]
[14, 21]
[15, 22]
[14, 23]
[14, 19]
[13, 18]
[12, 17]
[11, 16]
[10, 15]
[9, 14]
[8, 14]
[7, 14]
[6, 15]
[5, 15]
[4, 16]
[3, 15]
[2, 14]
[2, 15]
[3, 16]
[5, 16]
[7, 15]
[8, 15]
[9, 15]
[8, 16]
[7, 17]
[9, 16]
[10, 16]
[9, 17]
[11, 18]
[10, 18]
[9, 19]
[8, 19]
[10, 19]
[12, 19]
[15, 33]
[16, 32]
[17, 21]
[16, 20]
[18, 29]
[18, 30]
[20, 24]
[20, 25]
[3, 14]
[8, 16]
[9, 13]
[9, 17]
[12, 16]
[13, 17]
[14, 18]
[15, 24]
[16, 20]
[18, 21]
[19, 22]
[3, 13]
[2, 12]
[8, 16]
[9, 12]
[9, 17]
[10, 18]
[11, 14]
[12, 15]
[13, 16]
[16, 34]
[17, 19]
[17, 33]
[19, 21]
[19, 29]
[19, 30]
[21, 24]
[21, 25]
[21, 26]
[2, 11]
[3, 12]
[4, 13]
[9, 11]
[8, 10]
[7, 11]
[6, 10]
[5, 9]
[4, 8]
[3, 7]
[5, 10]
[4, 11]
[3, 11]
[5, 11]
[6, 11]
[5, 12]
[14, 16]
[17, 18]
[17, 34]
[19, 20]
[18, 19]
[19, 21]
[20, 21]
[2, 7]
[1, 8]
[0, 9]
[3, 11]
[4